In [4]:
from rdflib import Graph
from rdflib import RDF
import sys
sys.path.append('../')

sys.path.append('../andre')
import andre.utils as utils
from andre.utils import schema as SCHEMA
import collections

# editor alignment BLTF/ BNF_Constellations

In [10]:
publishers_btlf_graph = Graph()
publishers_btlf_graph.bind("schema","http://schema.org/", override=True, replace=True)
publishers_btlf_graph.parse("../final_datasets/btlf_books/Graphes/grapheEditeurs_Imprint_andre.ttl")
print(len(list(publishers_btlf_graph.subjects(RDF.type, SCHEMA.Publisher))))


publishers_bnf_constellations_graph = Graph()
publishers_bnf_constellations_graph.bind("schema","http://schema.org/", override=True, replace=True)
publishers_bnf_constellations_graph.parse("../final_datasets/publisher_BNF_Constellation_NoDuplicate.ttl")
len(list(publishers_bnf_constellations_graph.subjects(RDF.type, SCHEMA.Publisher)))

1022


2239

In [11]:
import Levenshtein

def find_top_x_similar_strings_levenshtein(string_list, target_string, top_x=5):
    distances = []
    for string in string_list:
        distance = Levenshtein.distance(string, target_string)
        if distance <= 8:
            distances.append((string, distance))
    distances.sort(key=lambda x: x[1])
    if distances:
         return distances[:top_x]
    else: 
        return None

In [12]:
publishers_bnf_constellations_dict = {}

for publisher in publishers_bnf_constellations_graph.subjects(RDF.type, SCHEMA.Publisher):
    raw_name = str(publishers_bnf_constellations_graph.value(publisher, SCHEMA.name))
    preprocessed_name = utils.preprocess_publisher_name(raw_name)
    
    if preprocessed_name not in publishers_bnf_constellations_dict.keys(): # don't add doublons
        # doublons[preprocessed_name]+= publisher
        publishers_bnf_constellations_dict[preprocessed_name] = utils.Publisher(source="bnf_constellations", 
                                                                        uri=publisher, 
                                                                        raw_name=raw_name, 
                                                                        preprocessed_name=preprocessed_name)
    else:
        print("duplicate found: check source file or eliminate")

In [13]:
len(publishers_btlf_graph)

2044

In [31]:
manual_matches = {"Gulf Stream (Editions du)": "Gulf_Stream", 
                  "Magnard-Vuibert": "Magnard",
                  "Musée du Louvre": "MuseduLouvred",
                 "Soleil productions": "Soleilproduction",
                 "Editions Ouest-France-Edilarge": "OuestFrance",
                 "Plume de carotte": "Plumecarotte",
                 "Editions Dadoclem": "Dadoclemd",
                 "Jean-Claude Gawsewitch éditeur": "JCGawsewitch",
                 "Editions de L'Archipel": "LArchipel",
                 "Odile Jacob": "OJacob",
                 "Editions du Centre Pompidou": "CentrePompidou",
                 "Musées de la ville de Strasbourg": "lesMusesdelaVilledeStrasbourg",
                 "Lancosme multimédia": "Lancosmed",
                 "Critères": "Critresd",
                 "Quespire": "Quespired",
                 "Les éditions du Bonhomme vert": "leBonhommevert",
                 "Vengeur masqué": "Le_Vengeur_Masqué",
                 "Editions Ecole active": "Ecole_Active",
                 "Editions [MiC_MaC]": "MicMac",
                 "Eveil et découvertes": "Eveiletdcouverte",
                 "Editions des Grandes personnes": "Les_Grandes_Personnes",
                 "Editions-Diffusion Karthala": "Karthala",
                 "Les éditions Carpentier": "DCarpentier",
                 "Pascal Galodé éditeurs": "PGalod",
                 "Prisma Media": "Prisma",
                 "Art à la page": "lArtlapage",
                 "CERCIL": "duCercil",
                 "Les Editions de la Grande ourse": "La_Grande_Ourse",
                 "Editions de La Différence": "La_Différence",
                 "Tertium": "Tertiumd",
                 "Editions de la Renarde rouge": "La_Renarde_Rouge",
                 "Aniss éditions": "Anissd",
                 "Editions du Rouergue": "Rouergue",
                 "Les éditions du soleil de minuit": "Soleil_De_Minuit",
                 "Editions des Arènes": "Les_Arènes",
                 "Le Square éditeur": "leSquared",
                 "Griffon bleu": "leGriffonbleu",
                 "Les éditions du Kilowatt": "Kilowatt",
                 "Ophelbe": "Ofelbe",
                 "Editions Privat SAS": "Privat",
                 "Losange": "duLosange",
                 "Les Editions Sandawe": "Sandawecom",
                 "Nouvelles éditions ivoiriennes": "NEINouvellesditionsIvoiriennes",
                 "Studio des Trois Becs": "StudiolesTroisBecs",
                 "Editions Imago": "AuzasditeursImago",
                 "205 Corp": "205fr",
                 "Librairie Téqui (Pierre)": "Téqui",
                 "Editions de la Coopérative": "laCooprative",
                 "Les Productions Somme toute": "Somme_toute",
                 "Réseau Canopé": "Canopditions",
                 "Editions Maelström": "maelstrmreEvolution",
                 "Le Dilettante": "Le_dilletante",
                 "Les Editions de Minuit": "Minuit",
                 "Editions Jean-Paul Gisserot": "JPGisserot",
                 "Les éditions du dernier havre": "Le_dernier_havre",
                 "Éditions du Septentrion": "Septentrion",
                 "Orca Book Publishing": "Orca",
                 "LéR - Les éditeurs réunis": "Les_Éditeurs_Réunis",
                 "Presses de l'Université de Montréal": "Les_Presses_De_Luniversité_De_Montréal",
                 "Éditions du Noroit": "Le_Noroît",
                 "Editions de Fallois": "Fallois",
                 "6 pieds sous terre éditions": "Six_Pieds_Sous_Terre",
                 "Editions du Triomphe": "Triomphe",
                 "Éditions du Vermillon": "Vermillon",
                 "Éditions ADA inc.": "AdA",
                 "Homme sans nom (Editions de l')": "LHomme_Sans_Nom",
                 "Editions des Mille saisons": "Mille_Saisons",
                 "Vincze (Eva)": "Vincze",
                 "Edizioni Corraini": "Corraini",
                 "Editions Picquier (Philippe)": "PPicquier"
                 }

direct_match_counter = 0
manual_match_counter = 0
levenstein_match_counter = 0
no_match_counter = 0
total_counter = 0

btlf_publishers_dict = {}
matches_dict = {}
non_matches_dict = {}
doublons_btlf = []
for publisher in publishers_btlf_graph.subjects(RDF.type, SCHEMA.Publisher):
    # print(publisher)
    total_counter += 1
    raw_name = str(publishers_btlf_graph.value(publisher, SCHEMA.name))
    preprocessed_name = utils.preprocess_publisher_name(raw_name)

    if preprocessed_name in btlf_publishers_dict.keys():
        doublons_btlf.append(preprocessed_name)
        
    btlf_publishers_dict[preprocessed_name] = utils.Publisher(source="btlf", 
                                                uri=publisher, 
                                                raw_name=raw_name, 
                                                preprocessed_name=preprocessed_name)

    if preprocessed_name in publishers_bnf_constellations_dict.keys():
        direct_match_counter +=1
        # print(preprocessed_name)
        matches_dict[preprocessed_name] = [btlf_publishers_dict[preprocessed_name], publishers_bnf_constellations_dict[preprocessed_name]]
    elif raw_name in manual_matches.keys(): # don't change graph
        # but: avoir uri du truc matché
        manual_match_counter += 1
        matched_uri = manual_matches[raw_name]
        print(raw_name)
        print(matched_uri)
        print(SCHEMA[matched_uri])
        # print(find_preprocessed_name_by_uri(SCHEMA[matched_uri]))
    else:
        no_match_counter += 1
        matched_key = find_top_x_similar_strings_levenshtein(publishers_bnf_constellations_dict.keys(), preprocessed_name)
        if matched_key:
            levenstein_match_counter += 1
            # print("\""+ raw_name+ "\"")
            # for key in matched_key:
            #     print("  ", publishers_bnf_constellations_dict[key[0]].uri.replace("http://schema.org/", " "))
        else:
            non_matches_dict[preprocessed_name] = publisher
            # no_match_counter += 1
print()
print("direct", direct_match_counter, "/", len(btlf_publishers_dict.keys()))
print("manual", manual_match_counter, "/", len(btlf_publishers_dict.keys()))
print("no match", no_match_counter, "/", len(btlf_publishers_dict.keys()))
print("levenstein", levenstein_match_counter, "/", len(btlf_publishers_dict.keys()))
print("total counter", total_counter)
print("doublons counter", len(doublons_btlf))
print("total number of keys in dict", total_counter - len(doublons_btlf))
print("sum = ", direct_match_counter  + no_match_counter, "==", len(btlf_publishers_dict.keys()))

Éditions du Vermillon
Vermillon
http://schema.org/Vermillon
Éditions ADA inc.
AdA
http://schema.org/AdA
Plume de carotte
Plumecarotte
http://schema.org/Plumecarotte
Musées de la ville de Strasbourg
lesMusesdelaVilledeStrasbourg
http://schema.org/lesMusesdelaVilledeStrasbourg
Critères
Critresd
http://schema.org/Critresd
Quespire
Quespired
http://schema.org/Quespired
Vengeur masqué
Le_Vengeur_Masqué
http://schema.org/Le_Vengeur_Masqué
Eveil et découvertes
Eveiletdcouverte
http://schema.org/Eveiletdcouverte
Tertium
Tertiumd
http://schema.org/Tertiumd
Les éditions du soleil de minuit
Soleil_De_Minuit
http://schema.org/Soleil_De_Minuit
Studio des Trois Becs
StudiolesTroisBecs
http://schema.org/StudiolesTroisBecs
Les éditions du dernier havre
Le_dernier_havre
http://schema.org/Le_dernier_havre
Éditions du Septentrion
Septentrion
http://schema.org/Septentrion
Orca Book Publishing
Orca
http://schema.org/Orca
LéR - Les éditeurs réunis
Les_Éditeurs_Réunis
http://schema.org/Les_Éditeurs_Réunis
Pr

In [32]:
doublons_btlf

['lesallusifs',
 'les400coups',
 'lemeac',
 'arte',
 'albinmicheljeunesse',
 'nordsud',
 'ventsdouest']

In [ ]:
len(non_matches_dict)

32

In [ ]:
non_matches_dict

{'canopecrdpdelacademiedeparis': rdflib.term.URIRef('http://schema.org/128'),
 'bretagneenvironnement': rdflib.term.URIRef('http://schema.org/150'),
 'maitrisedeshautsdeseine': rdflib.term.URIRef('http://schema.org/232'),
 'sncfdirectiondelarecherchedepartementmarketingetdeveloppement': rdflib.term.URIRef('http://schema.org/287'),
 'radiofranceinternationale': rdflib.term.URIRef('http://schema.org/298'),
 'canopecrdpdelacademiedaixmarseille': rdflib.term.URIRef('http://schema.org/301'),
 'associationculturellefranceamerique': rdflib.term.URIRef('http://schema.org/314'),
 'galeriebarbier&mathon': rdflib.term.URIRef('http://schema.org/316'),
 'museeduluxembourgdeparis': rdflib.term.URIRef('http://schema.org/334'),
 'lememorialdecaen': rdflib.term.URIRef('http://schema.org/363'),
 'chateaudeversailles': rdflib.term.URIRef('http://schema.org/395'),
 'mondeglobalnouvelles': rdflib.term.URIRef('http://schema.org/400'),
 'bibliothequelecturedeveloppement': rdflib.term.URIRef('http://schema.or